In [1]:
!nvidia-smi


Thu Nov 11 15:11:06 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P8    32W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ls
%cd gdrive/My\ Drive/TFKeras/
!ls

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.activity.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fexperimentsandconfigs%20https%3a%2f%2fwww.googleapis.com%2fauth%2fphotos.native&response_type=code

Enter your authorization code:
4/1AX4XfWj1hh0I9qQuJubRRAfOqt-xKAdvFuSom4-5QkkynahJRXqdFUI9r4M
Mounted at /content/gdrive
gdrive	sample_data
/content/gdrive/My Drive/TFKeras
began	      chap3.ipynb   chap6_a.ipynb  chap9.ipynb	model
chap10.ipynb  chap4.ipynb   chap6_b.ipynb  data		models
chap11.ipynb  chap5-cnn.py  chap7.ipynb    img		__py

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Fri Oct  1 00:05:33 2021
create network
@author: Administrator
"""
import time 
start = time.time()

#add tensorflow. before keras
import keras
from tensorflow.keras.models import Sequential  #序贯模型
from tensorflow.keras.layers import Dense    #全连接层
from tensorflow.keras.layers import Dropout  #随机失活层
from tensorflow.keras.layers import Flatten  #展平层，从卷积层到全连接层必须展平
from tensorflow.keras.layers import Conv1D   #卷积层
from tensorflow.keras.layers import MaxPooling1D  #最大值池化
import pandas as pd
from tensorflow.keras import backend as k
#from sklearn.cross_validation import trainbuzhiyunyun_test_split #随机划分为训练子集和测试子集
from sklearn.model_selection import train_test_split  #leon
#from keras.utils.vis_utils import plot_model #leon
from tensorflow.keras.utils import  plot_model  #leon
from tensorflow.keras.optimizers import SGD 
import matplotlib.pyplot as plt
import pydot #leon

batch_size = 128  #一批训练样本128张图片
num_classes = 40  #有40个类别
epochs = 2   #一共迭代12轮

# x_train = pd.read_csv('train_xe.csv',header=None).values
# y_train = pd.read_csv('train_ye.csv',header=None).values

# x_train = pd.read_csv('train_xe.csv',header=None)
# y_train = pd.read_csv('train_ye.csv',header=None)

x_train = pd.read_csv('train_2.csv',header=None)
y_train = pd.read_csv('test_2.csv',header=None)


print("x_train", x_train.head())
print("y_train", y_train.head())

# x_test = pd.read_csv('test_x.csv',header=None).values
# y_test = pd.read_csv('test_y.csv',header=None).values

# X = x_train.iloc[:, 1:42]
# Y = y_train.iloc[:, 0]

#从训练集中手动指定验证集
X_train, X_test, Y_train, Y_test = train_test_split(x_train, y_train, test_size=0.2, random_state=42)
#x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.15, random_state=2)
"""
print("shape", x_train.shape[0])
if k.image_data_format() == 'channels_first':
   x_train = x_train.reshape(x_train.shape[0], 1, 41)
   x_test = x_test.reshape(x_test.shape[0], 1, 41)
   #x_dev = x_dev.reshape(x_dev.shape[0], 1, 41)
   input_shape = (1, 41)
else:
   x_train = x_train.reshape(x_train.shape[0], 41, 1)
   x_test = x_test.reshape(x_test.shape[0], 41, 1)
   #x_dev = x_dev.reshape(x_dev.shape[0], 41, 1)
   input_shape = (41, 1)
"""

print(Y_train.shape)
print(Y_test.shape)

y2_train = Y_train.values.ravel()
y2_test  = Y_test.values.ravel()
print(y2_train.shape)
print(y2_test.shape)


#ensemble code
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier


from sklearn.svm import SVC
 
log_clf = LogisticRegression(solver='lbfgs', max_iter=500)
rnd_clf = RandomForestClassifier()
svm_clf = SVC()
nb_clf = GaussianNB()
dt_clf = DecisionTreeClassifier(random_state=0) 

# voting_clf=VotingClassifier(
#     estimators=[('lc',log_clf),('rf',rnd_clf,),('svc',svm_clf)],
#     voting='soft'
# )
 
voting_clf=VotingClassifier(
    estimators=[('lc',log_clf),('dt',dt_clf), ('nv', nb_clf), ('rf',rnd_clf), ('svm',svm_clf)],
    # voting='soft'chaoge 
    voting='soft'
)


from sklearn.metrics import accuracy_score

#y_train = y_train.reshape(y_train.shape[0], 1)
#y_train = y_train.iloc[:, 1]
#print("y shape:", y_train.shape)
#y_train = y_train[0]  #leon
#x_test = x_test.reshape(x_test.shape[0], 1, 41)   #leon


# for clf in (log_clf,rnd_clf,svm_clf,voting_clf):
#     clf.fit(X_train,y2_train)
#     y_pred=clf.predict(X_test)
#     print(clf.__class__.__name__,accuracy_score(y2_test, y_pred))

for clf in (log_clf, dt_clf, nb_clf, rnd_clf, svm_clf, voting_clf):
#for clf in (log_clf,rnd_clf):
    clf.fit(X_train,y2_train)
    y_pred=clf.predict(X_test)
    print(clf.__class__.__name__,accuracy_score(y2_test, y_pred))

"""

model = Sequential()  #sequential序贯模型:多个网络层的线性堆叠
#输出的维度（卷积滤波器的数量）filters=32；1D卷积窗口的长度kernel_size=3；激活函数activation   模型第一层需指定input_shape：
model.add(Conv1D(32, 3, activation='relu',input_shape=input_shape))  #data_format默认channels_last
model.add(MaxPooling1D(pool_size=(2))) #池化层：最大池化  池化窗口大小pool_size=2
model.add(Flatten())  #展平一个张量，返回一个调整为1D的张量
#model.add(Dropout(0.25))  #需要丢弃的输入比例=0.25    dropout正则化-减少过拟合
model.add(Dense(128, activation='relu',name='fully_connected')) #全连接层
model.add(Dense(num_classes, activation='softmax',name='softmax'))

#编译，损失函数:多类对数损失，用于多分类问题， 优化函数：adadelta， 模型性能评估是准确率
model.compile(loss='categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])
#运行 ， verbose=1输出进度条记录      epochs训练的轮数     batch_size:指定进行梯度下降时每个batch包含的样本数
model.fit(x_train, y_train, batch_size= batch_size, epochs=epochs, verbose=1)
#history = model.fit(x_train, y_train, batch_size= batch_size, epochs=epochs, verbose=0, validation_data=(x_dev, y_dev))
"""
"""
#模型的训练损失（loss）和验证损失（val_loss），以及训练准确率（acc）和验证准确率（val_acc）可以使用绘图代码绘制出来
def smooth_curve(points,factor=0.8): #定义使曲线变得平滑
    smoothed_points = []
    for point in points:
        if smoothed_points:
            previous = smoothed_points[-1]
            smoothed_points.append(previous * factor + point * (1 - factor))
        else:
            smoothed_points.append(point)
    return smoothed_points

loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs,loss, 'bo', label = 'Training loss')
plt.plot(epochs,val_loss, 'b', label = 'Validation loss')
plt.title('Training and validatio loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.figure()
acc = history.history['acc']
val_acc = history.history['val_acc']
plt.plot(epochs, acc,'bo', label = 'Training acc')
plt.plot(epochs, val_acc, 'b', label = 'Validation acc')
plt.title('Training and validatio accuracy')
plt.legend()
plt.show()
"""
"""
# 将测试集输入到训练好的模型中，查看测试集的误差
score = model.evaluate(x_test, y_test, verbose=0,batch_size= batch_size)
print('Test loss:', score[0])
print('Test accuracy: %.2f%%' % (score[1] * 100))

#运行的时间
stop = time.time()
print(str(stop-start) + "秒")

# 神经网络可视化
plot_model(model, to_file='D:/model21.png',show_shapes=True)

#输出模型各层参数情况
model.summary()
"""

FileNotFoundError: ignored